In [1]:
class Bucket:
    def __init__(self, depth, size):
        self.depth = depth
        self.size = size
        self.items = {}  # Dictionary for storing key-value pairs

    def is_full(self):
        return len(self.items) >= self.size

    def insert(self, key, value):
        self.items[key] = value

    def delete(self, key):
        if key in self.items:
            del self.items[key]

    def search(self, key):
        return self.items.get(key, None)

class ExtendibleHashTable:
    def __init__(self, bucket_size=2):
        self.global_depth = 1
        self.bucket_size = bucket_size
        self.directory = [Bucket(self.global_depth, bucket_size) for _ in range(2)]

    def _hash(self, key):
        return hash(key) & ((1 << self.global_depth) - 1)

    def insert(self, key, value):
        index = self._hash(key)
        bucket = self.directory[index]

        if key in bucket.items or not bucket.is_full():
            bucket.insert(key, value)
            print(f"Inserted ({key}, {value}) into bucket {index}")
            return

        # Bucket is full, need to split
        print(f"Bucket {index} is full. Splitting...")
        self._split_bucket(index)
        # Re-insert the key
        self.insert(key, value)

    def _split_bucket(self, index):
        old_bucket = self.directory[index]
        local_depth = old_bucket.depth
        old_bucket.depth += 1

        if old_bucket.depth > self.global_depth:
            self._double_directory()

        new_bucket = Bucket(old_bucket.depth, self.bucket_size)

        # Update directory to point to new buckets
        for i in range(len(self.directory)):
            if self.directory[i] is old_bucket and ((i >> local_depth) & 1):
                self.directory[i] = new_bucket

        # Rehash the keys in the old bucket
        all_items = list(old_bucket.items.items())
        old_bucket.items.clear()

        for k, v in all_items:
            self.insert(k, v)  # Will hash and reinsert into appropriate bucket

    def _double_directory(self):
        print("Doubling directory size.")
        self.directory += self.directory
        self.global_depth += 1

    def search(self, key):
        index = self._hash(key)
        value = self.directory[index].search(key)
        if value is not None:
            print(f"Found key {key} with value {value} in bucket {index}")
        else:
            print(f"Key {key} not found.")
        return value

    def delete(self, key):
        index = self._hash(key)
        bucket = self.directory[index]
        if key in bucket.items:
            bucket.delete(key)
            print(f"Key {key} deleted from bucket {index}")
        else:
            print(f"Key {key} not found for deletion.")

    def display(self):
        seen = set()
        print("\nDirectory:")
        for i, bucket in enumerate(self.directory):
            if id(bucket) not in seen:
                seen.add(id(bucket))
                print(f"Bucket {i} (depth={bucket.depth}): {bucket.items}")
                
ht = ExtendibleHashTable(bucket_size=2)

ht.insert(1, "One")
ht.insert(2, "Two")
ht.insert(3, "Three")
ht.insert(4, "Four")
ht.insert(5, "Five")

ht.display()

ht.search(3)
ht.delete(3)
ht.search(3)

ht.display()


Inserted (1, One) into bucket 1
Inserted (2, Two) into bucket 0
Inserted (3, Three) into bucket 1
Inserted (4, Four) into bucket 0
Bucket 1 is full. Splitting...
Doubling directory size.
Inserted (1, One) into bucket 1
Inserted (3, Three) into bucket 3
Inserted (5, Five) into bucket 1

Directory:
Bucket 0 (depth=1): {2: 'Two', 4: 'Four'}
Bucket 1 (depth=2): {1: 'One', 5: 'Five'}
Bucket 3 (depth=2): {3: 'Three'}
Found key 3 with value Three in bucket 3
Key 3 deleted from bucket 3
Key 3 not found.

Directory:
Bucket 0 (depth=1): {2: 'Two', 4: 'Four'}
Bucket 1 (depth=2): {1: 'One', 5: 'Five'}
Bucket 3 (depth=2): {}
